In [ ]:
# Import necessary libraries
import cv2
from pathlib import Path
from PIL import Image, ImageSequence
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
from numpy.fft import fft2, fftshift, ifft2, ifftshift
from numpy import unravel_index
import pandas as pd

# Function to convert TIFF images to PNG format
def tiff_to_png(input_tiff, input_path, output_path):
    # Attempt to open and process the TIFF file
    try:
        # Open the TIFF image
        sq = Image.open(os.path.join(input_path, input_tiff))
        # Iterate through each frame in the TIFF stack
        for i, img in enumerate(ImageSequence.Iterator(sq)):
            # Define the output file path for each frame
            output = os.path.join(output_path, f"frame_{i:06d}.png")
            # Save the frame as a PNG file
            img.save(output)
    # Ensure all resources are freed after processing
    finally:
        print("PNG extraction done")

# Define input and output paths
input_path = '/content/drive/input_video'
input_tiff = "Video.tif"
output_base_path = "/content/drive/output_path"
output_path = Path(output_base_path) / Path(input_tiff).stem
output_path.mkdir(parents=True, exist_ok=True)

# Convert TIFF to PNG
tiff_to_png(input_tiff, input_path, output_path)

# Define path for saving results
result_folder = "/content/drive/results" + Path(input_tiff).stem + "_Results_bw_"
Path(result_folder).mkdir(parents=True, exist_ok=True)

# Initialize error log list
error_log = []

# Process each PNG file generated from the TIFF
for filename in os.listdir(output_path):
    file_path = os.path.join(output_path, filename)
    image = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)

    # Create a figure with 2x3 subplot arrangement
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    axes = axes.ravel()

    # Display the original image
    axes[0].imshow(image, cmap='gray', vmin=0, vmax=255)
    axes[0].set_title('Original Image')

    # Apply Fourier Transform to the image and display it
    ftimage = fft2(image)
    ftimage = fftshift(ftimage)
    axes[1].imshow(20 * np.log(np.abs(ftimage) + 1), cmap='gray', vmin=0, vmax=255)
    axes[1].set_title('Fourier Transform')

    # Create a bandpass filter mask and apply it
    rows, cols = ftimage.shape
    crow, ccol = rows // 2, cols // 2
    mask = np.zeros((rows, cols), dtype=np.uint8)
    r_in, r_out = 6, 9
    x, y = np.ogrid[:rows, :cols]
    mask_area = np.logical_and(((x - crow)**2 + (y - ccol)**2 >= r_in**2),
                               ((x - crow)**2 + (y - ccol)**2 <= r_out**2))
    mask[mask_area] = 1
    axes[2].imshow(mask, cmap='gray')
    axes[2].set_title('Bandpass Filter')

    # Apply mask to the Fourier-transformed image, inverse transform it, and display
    m_app_ftimage = ftimage * mask
    i_ftimage = ifftshift(m_app_ftimage)
    result_img = ifft2(i_ftimage)
    tmp = np.log(np.abs(result_img) + 1)
    axes[3].imshow(tmp, cmap='gray', vmin=tmp.min(), vmax=tmp.max())
    axes[3].set_title('Filtered Image')

    # Plot the intensity profile of the filtered image
    axes[4].plot(tmp[int(tmp.shape[0]/2), :], color='blue')
    axes[4].set_title('Intensity Profile')

    # Detect peaks in the intensity profile to determine the ring structure
    crow, ccol = unravel_index(tmp.argmax(), tmp.shape)
    central_line_y = tmp[crow, :]
    central_line_x = tmp[:, ccol]

    # Find peaks in the intensity profiles along both directions
    peaks_y, _ = find_peaks(central_line_y, height=0.1)
    peaks_x, _ = find_peaks(central_line_x, height=0.1)

    # Determine if valid peaks exist and calculate the radius of rings
    if len(peaks_x) > 1 and len(peaks_y) > 1:
        closest_peak_x = min(peaks_x, key=lambda x: abs(x-ccol))
        closest_peak_y = min(peaks_y, key=lambda y: abs(y-crow))

        peak_index_x = peaks_x.tolist().index(closest_peak_x)
        peak_index_y = peaks_y.tolist().index(closest_peak_y)

        if peak_index_x > 0 and peak_index_y > 0:
            rl_x = ccol - peaks_x[peak_index_x - 2]
            rr_x = peaks_x[peak_index_x + 2] - ccol
            ru_y = crow - peaks_y[peak_index_y - 2]
            rl_y = peaks_y[peak_index_y + 2] - crow

            circle_radius = round((rl_x + rr_x + ru_y + rl_y) / 4., 0)
            circle_img = cv2.circle(image.copy(), (ccol, crow), int(circle_radius), (255, 0, 0), 2)

            axes[5].imshow(circle_img, cmap='gray', vmin=0, vmax=255)
            axes[5].set_title('Result with Circle')
            plt.tight_layout()
            plt.savefig(os.path.join(result_folder, f"composite_{filename}"))
            plt.show()
        else:
            print(f"Skipped frame {filename} due to invalid peak bounds.")
            plt.close(fig)
    else:
        print(f"Skipped frame {filename} due to no valid peaks.")
        plt.close(fig)

print("All images processed and displayed.")
